In [1]:
from tkinter import *
from PIL import ImageTk, Image

In [65]:
class GUI(Frame):
    def __init__(self,master):
        Frame.__init__(self,master)
        self.master=master
        self.btn_arr=[[0 for i in range(8)] for j in range(8)]
        self.has_moven=[[False for i in range(16)],[False for i in range(16)]]
        self.chance,self.ind=0,None
        
        self.bg_color=['#C0C0C0','#404040']
        self.mv_color=['#99D9EA','#59C2DD']
        self.kl_color=['#FF7F27','#F26100']
        self.sl_color=['#FFF200','#FFB90E']
        self.piece_name='rnbqkbnrpppppppp'
        self.piece_arr=self.create_pos()
        self.load_main_page()
        self.set_pieces()
        self.moves=move_class(self.piece_arr)
        self.move,self.kill,self.save,self.hidden=self.moves.initialise_all(self.chance,self.has_moven)
        
    def create_pos(self):
        row_order=[[0,1],[7,6]]
        pos_arr=[[],[]]
        for color in range(2):
            for col in range(16):
                pos_arr[color].append((row_order[color][col//8],col%8))
        return pos_arr
    
    def set_pieces(self):
        for color in [self.chance,self.chance^1]:
            for ind,pos in enumerate(self.piece_arr[color]):
                x,y=pos
                img=ImageTk.PhotoImage(Image.open("{}{}.png".format(self.piece_name[ind],color)))
                self.btn_arr[x][y].configure(image=img,width=70,height=70)
                self.btn_arr[x][y].image=img
                                       
        
    def load_main_page(self):
        self.master.title('The game of CHESS')
        self.master.configure(background='black')
        self.master.geometry("570x570")
        self.master.resizable(0, 0)
        
        for i in range(8):
            for j in range(8):
                img = ImageTk.PhotoImage(Image.open("transparent_bg.png"))
                self.btn_arr[i][j] = Button(self.master, image=img, background=self.bg_color[(i+j)%2],
                                            name="({},{})".format(i,j))
                self.btn_arr[i][j].image=img
                self.btn_arr[i][j].bind("<Button-1>", self.onClick)
                self.btn_arr[i][j].place(x=2+70*j,y=2+70*i)
                
    def colourise(self,x,y):
        self.btn_arr[x][y].configure(bg=self.sl_color[(x+y)%2])
        for i,j in self.move[self.chance][self.ind]:
            self.btn_arr[i][j].configure(bg=self.mv_color[(i+j)%2])
        for i,j in self.kill[self.chance][self.ind]:
            self.btn_arr[i][j].configure(bg=self.kl_color[(i+j)%2])
            
    def decolourise(self,ind):
        x,y=self.piece_arr[self.chance][ind]
        self.btn_arr[x][y].configure(bg=self.bg_color[(x+y)%2])
        for i,j in self.move[self.chance][ind]:
            self.btn_arr[i][j].configure(bg=self.bg_color[(i+j)%2])
        for i,j in self.kill[self.chance][ind]:
            self.btn_arr[i][j].configure(bg=self.bg_color[(i+j)%2])
            
    def piece_shift(self,ind,x,y):
        i,j=self.piece_arr[self.chance][ind]
        img=ImageTk.PhotoImage(Image.open("transparent_bg.png"))
        self.btn_arr[i][j].configure(image=img,width=70,height=70)
        self.btn_arr[i][j].image=img
        self.piece_arr[self.chance][ind]=(x,y)
        img=ImageTk.PhotoImage(Image.open("{}{}.png".format(self.piece_name[ind],self.chance)))
        self.btn_arr[x][y].configure(image=img,width=70,height=70)
        self.btn_arr[x][y].image=img
                
    def onClick(self,event):
        x,y=eval(event.widget.winfo_name())
        if self.ind==None and ((x,y) in self.piece_arr[self.chance]):
            # only checking on pieces depending on self.chance
            for ind,pos in enumerate(self.piece_arr[self.chance]):
                if pos==(x,y):
                    self.ind=ind
                    print(self.ind)
                    print('Selected object \n move:{},kill:{},safe={},hidden={}'
                          .format(self.move[self.chance][self.ind],self.kill[self.chance][self.ind],
                                  self.save[self.chance][self.ind],self.hidden[self.chance][self.ind]))
            self.colourise(x,y)

        # If something is selected earlier and now clicked in one of its moveable positions execute below
        elif self.ind!=None and ((x,y) in self.move[self.chance][self.ind].union(self.kill[self.chance][self.ind])):
            if ((x,y) in self.kill[self.chance][self.ind]):
                for i,j in enumerate(self.piece_arr[self.chance^1]):
                    if j==(x,y):
                        self.piece_arr[self.chance^1][i]=None
                        break
                        
            self.decolourise(self.ind)
            self.piece_shift(self.ind,x,y)
            self.has_moven[self.chance][self.ind]=True
            self.chance^=1
            self.move,self.kill,self.save,self.hidden=self.moves.initialise_all(self.chance,self.has_moven)
            self.ind=None
        elif self.ind!=None:
            self.decolourise(self.ind)
            self.ind=None

In [64]:
root=Tk()
app=GUI(root)
app.mainloop()

12
Selected object 
 move:{(3, 4), (2, 4)},kill:set(),safe=set(),hidden={(2, 5), (2, 3)}
13
Selected object 
 move:{(4, 5), (5, 5)},kill:set(),safe=set(),hidden={(5, 6), (5, 4)}
12
Selected object 
 move:{(4, 4)},kill:{(4, 5)},safe=set(),hidden={(4, 3)}
12
Selected object 
 move:{(5, 4), (4, 4)},kill:set(),safe=set(),hidden={(5, 5), (5, 3)}
12
Selected object 
 move:{(5, 5)},kill:{(5, 4)},safe=set(),hidden={(5, 6)}
11
Selected object 
 move:{(4, 3), (5, 3)},kill:{(5, 4)},safe=set(),hidden={(5, 2)}


In [13]:
class move_class:
    def __init__(self,obj_arr):
        self.obj_arr=obj_arr
        self.move=[[set() for i in range(16)],[set() for i in range(16)]]
        self.kill=[[set() for i in range(16)],[set() for i in range(16)]]
        self.save=[[set() for i in range(16)],[set() for i in range(16)]]
        self.hidden=[[set() for i in range(16)],[set() for i in range(16)]]
    
    def initialise_all(self,chance,has_moven):
        self.has_moven=has_moven
        for color in [chance,chance^1]:
            for ind,pos in enumerate(self.obj_arr[color]):
                if pos!=None:
                    if ind in [0,7]: t1,t2,t3,t4=self.QBRN(pos,color,self.obj_arr,
                                        direction_vect=[(1,0),(0,1),(-1,0),(0,-1)],extend=8)
                    elif ind in [1,6]: t1,t2,t3,t4=self.QBRN(pos,color,self.obj_arr,
                                        direction_vect=[(2,1),(2,-1),(-2,1),(-2,-1),(1,2),(-1,2),(1,-2),(-1,-2)],extend=2)   
                    elif ind in [2,5]: t1,t2,t3,t4=self.QBRN(pos,color,self.obj_arr,
                                        direction_vect=[(1,1),(-1,1),(-1,-1),(1,-1)],extend=8)
                    elif ind==3: t1,t2,t3,t4=self.QBRN(pos,color,self.obj_arr,
                                        direction_vect=[(1,0),(0,1),(-1,0),(0,-1),(1,1),(-1,1),(-1,-1),(1,-1)],extend=8)
                    elif ind==4: t1,t2,t3,t4=self.king(pos,color,self.obj_arr,
                                        direction_vect=[(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)],extend=2)
                    elif ind in [8,9,10,11,12,13,14,15]: t1,t2,t3,t4=self.pawn(pos,color,self.obj_arr,ind)
                    self.move[color][ind]=t1
                    self.kill[color][ind]=t2
                    self.save[color][ind]=t3
                    self.hidden[color][ind]=t4
        return self.move,self.kill,self.save,self.hidden

    def QBRN(self,pos,color,arr,direction_vect,extend):
        x,y=pos
        move,save,kill,hidden=set(),set(),set(),set()
        for i,j in direction_vect:
            save_kill_flag=False
            for k in range(1,extend):
                npos_x,npos_y=x+k*i,y+k*j
                if npos_x<0 or npos_y<0 or npos_y>7 or npos_x>7:
                    break
                if save_kill_flag==False and not ((npos_x,npos_y) in arr[0] or 
                                                  (npos_x,npos_y) in arr[1]):
                    move.add((npos_x,npos_y))
                elif save_kill_flag==False and (npos_x,npos_y) in arr[color]:
                    save_kill_flag=True
                    save.add((npos_x,npos_y))
                elif save_kill_flag==False and (npos_x,npos_y) in arr[color^1]:
                    save_kill_flag=True
                    kill.add((npos_x,npos_y))
                elif save_kill_flag==True:
                    hidden.add((npos_x,npos_y))
#         if not (check_arr==None or len(check_arr)==0):
#             move.intersection_update(check_arr)
#             kill.intersection_update(check_arr)
#         if not (pieces==None or len(pieces)==0) and pos in pieces:
#             move,kill=set(),set()
        return move,kill,save,hidden
    
    def king(self,pos,color,arr,direction_vect,extend):
        move,kill,save,hidden=self.QBRN(pos,color,arr,direction_vect,extend)
        for ind,moves in enumerate(self.move[color^1][:8]):
            if ind==4:
                hidden.update(move.intersection(self.hidden[color^1][4]))
                hidden.update(move.intersection(self.move[color^1][4]))
                move.difference_update(self.hidden[color^1][4])
            else:
                intersect=move.intersection(moves)
                move.difference_update(intersect)
                hidden.update(intersect)
            
        for ind,hiddens in enumerate(self.hidden[color^1][8:]):
            intersect=move.intersection(hiddens)
            move.difference_update(intersect)
            hidden.update(intersect)
            
        for i in self.save[color^1]:
            intersect=kill.intersection(i)
            kill.difference_update(intersect)
            hidden.update(intersect)
            
#         check_arr,pieces=set(),set()
#         for ind,i in self.obj_arr[color^1]:
#             if pos in i.kill:
#                 if i.name in ["pawn","knight"]:
#                     self.check_arr.update([i.pos])
#                 if i.name in ["rook","bishop","queen"]:
#                     self.check_arr.update(self.RBQ_check_arr_update(self.pos,i.pos))
#             if (self.pos in i.hidden) and (i.name in ["rook","bishop","queen"]):
#                 temp=self.RBQ_check_arr_update(self.pos,i.pos)
#                 if len(temp)==0:
#                     continue
#                 status=False
#                 for j in temp[1:]:
#                     for k in obj[self.color]:
#                         if k.pos in temp:
#                             self.pieces.add(j)
#                             status=True
#                             break
#                     for k in obj[(self.color+1)%2]:
#                         if k.pos in temp:
#                             status=True
#                             break
#                     if status==True:
#                         break
        return move,kill,save,hidden
    
    def pawn(self,pos,color,arr,ind):
        x,y=pos
        move,kill,save,hidden=set(),set(),set(),set()
        sign_bit=1-2*(color)
        if (x+sign_bit,y) not in arr[0]+arr[1]:
            move={(x+sign_bit,y)}
            if self.has_moven[color][ind]==False and (x+2*sign_bit,y) not in arr[0]+arr[1]:
                move.add((x+2*sign_bit,y))
        if y>=1:
            if (x+sign_bit,y-1) not in arr[0]+arr[1]:
                hidden.add((x+sign_bit,y-1))
            elif (x+sign_bit,y-1) in arr[color]:
                save.add((x+sign_bit,y-1))
            elif (x+sign_bit,y-1) in arr[color^1]:
                kill.add((x+sign_bit,y-1))
        if y<=6:
            if (x+sign_bit,y+1) not in arr[0]+arr[1]:
                hidden.add((x+sign_bit,y+1))
            elif (x+sign_bit,y+1) in arr[color]:
                save.add((x+sign_bit,y+1))
            elif (x+sign_bit,y+1) in arr[color^1]:
                kill.add((x+sign_bit,y+1))
#         if not (check_arr==None or len(check_arr)==0):
#             move.intersection_update(check_arr)
#             kill.intersection_update(check_arr)
#         if not (pieces==None or len(pieces)==0) and pos in pieces:
#             move,kill=set(),set()
        return move,kill,save,hidden

In [67]:
print(dir(app))

['_Misc__winfo_getint', '_Misc__winfo_parseitem', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_bind', '_configure', '_displayof', '_do', '_getboolean', '_getconfigure', '_getconfigure1', '_getdoubles', '_getints', '_grid_configure', '_gridconvvalue', '_last_child_ids', '_name', '_nametowidget', '_noarg_', '_options', '_register', '_report_exception', '_root', '_setup', '_subst_format', '_subst_format_str', '_substitute', '_tclCommands', '_w', '_windowingsystem', 'after', 'after_cancel', 'after_idle', 'anchor', 'bbox', 'bell', 'bg_color', 'bind', 'bind_all', 'bind_class', 'bindtags', 'btn_arr', 'cget', 'chance', 'children', 'clipboard_append', 'clipboard_clear',